<a href="https://colab.research.google.com/github/Abdulnihal/machinelearning/blob/main/machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

Hello Google Drive!

In [18]:

import glob
dataset=glob.glob("drive/My Drive/content/drive/MyDrive/dataset/images")


In [21]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [60]:
train_dir = '/content/drive/MyDrive/dataset/images/Train'
test_dir = '/content/drive/MyDrive/dataset/images/Test'

In [70]:
train_size = len(os.listdir('/content/drive/MyDrive/dataset/images/Train'))
test_size = len(os.listdir('/content/drive/MyDrive/dataset/images/Test'))

train_size,test_size

(576, 44)

In [71]:
df = pd.read_csv('/content/drive/MyDrive/dataset/players.csv')
df.head()

,Unnamed: 0,image,player
0,0,eda97f557b.jpg,virat_kohli
1,1,273ed0f2b9.jpg,hardik_pandya
2,2,77076b6409.jpg,jasprit_bumrah
3,3,f028e2dcf8.jpg,mohammed_shami
4,4,3364f861e1.jpg,kedar_jadhav


In [72]:
p_names = sorted(df['player'].unique())
n_classes = len(p_names)
print(n_classes)
p_names

15


['bhuvneshwar_kumar',
 'dinesh_karthik',
 'hardik_pandya',
 'jasprit_bumrah',
 'k._l._rahul',
 'kedar_jadhav',
 'kuldeep_yadav',
 'mohammed_shami',
 'ms_dhoni',
 'ravindra_jadeja',
 'rohit_sharma',
 'shikhar_dhawan',
 'vijay_shankar',
 'virat_kohli',
 'yuzvendra_chahal']

In [73]:
#Converting classes to numbers
class_to_num = dict(zip(p_names,range(n_classes)))

In [74]:
#Function to load and convert images to array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical

def images_to_array(data_dir,df,image_size):
    image_names = df['image']
    image_labels = df['player']
    data_size = len(image_names)
    
    X = np.zeros([data_size,image_size[0],image_size[1],image_size[2]],dtype = np.uint8)
    y = np.zeros([data_size,1],dtype = np.uint8)
    
    for i in range(data_size):
        img_name = image_names[i]
        img_dir = os.path.join(data_dir,img_name)
        img_pixels = load_img(img_dir,target_size=image_size)
        X[i] = img_pixels
        y[i] = class_to_num[image_labels[i]]
        
    y = to_categorical(y)
    
    ind = np.random.permutation(data_size)
    X = X[ind]
    y = y[ind]
    print('Ouptut Data Size: ', X.shape)
    print('Ouptut Label Size: ', y.shape)
    return X, y   

In [75]:
#Selecting image size according to pretrained models
img_size = (299,299,3)

In [76]:
X, y = images_to_array(train_dir,df,img_size)

Ouptut Data Size:  (576, 299, 299, 3)
Ouptut Label Size:  (576, 15)


In [77]:
#Function to extract features from images
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D,Lambda, Dropout, InputLayer, Input

def get_features(model_name, data_preprocessor, input_size, data):
    #Prepare pipeline.
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    #Extract feature.
    feature_maps = feature_extractor.predict(data, batch_size=32, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [78]:
#Extracting features using InceptionV3
from keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input
inception_features = get_features(InceptionV3,
                                  inception_preprocessor,
                                  img_size, X)

18/18 [==============================] - 117s 6s/step
Feature maps shape:  (576, 2048)


In [79]:
#Extracting features using Xception
from keras.applications.xception import Xception, preprocess_input
xception_preprocessor = preprocess_input
xception_features = get_features(Xception,
                                 xception_preprocessor,
                                 img_size, X)

18/18 [==============================] - 219s 12s/step
Feature maps shape:  (576, 2048)


In [80]:
#Extracting features using InceptionResnetV2
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
inc_resnet_preprocessor = preprocess_input
inc_resnet_features = get_features(InceptionResNetV2,
                                   inc_resnet_preprocessor,
                                   img_size, X)

18/18 [==============================] - 273s 15s/step
Feature maps shape:  (576, 1536)


In [81]:
#concatinating features
final_features = np.concatenate([inception_features,
                                 xception_features,
                                 inc_resnet_features,], axis=-1)
print('Final feature maps shape', final_features.shape)

Final feature maps shape (576, 5632)


In [ ]:

del X


In [84]:
#Callbacks
from keras.callbacks import EarlyStopping
EarlyStop_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
my_callback=[EarlyStop_callback]


In [91]:
#Building Model
from keras.models import Sequential
model = Sequential()
model.add(InputLayer(final_features.shape[1:]))
model.add(Dropout(0.7))
model.add(Dense(15,activation='softmax'))

In [92]:
#Compiling Model
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [93]:
#Training Model
history = model.fit(final_features,
                  y,
                  batch_size=32,
                  epochs=50,
                  validation_split=0.1,
                  callbacks=my_callback)

Epoch 1/50
17/17 [==============================] - 1s 21ms/step - loss: 3.2434 - accuracy: 0.0695 - val_loss: 2.4308 - val_accuracy: 0.1379
Epoch 2/50
17/17 [==============================] - 0s 8ms/step - loss: 2.5886 - accuracy: 0.2066 - val_loss: 2.2810 - val_accuracy: 0.2241
Epoch 3/50
17/17 [==============================] - 0s 8ms/step - loss: 2.2402 - accuracy: 0.2620 - val_loss: 2.2175 - val_accuracy: 0.2414
Epoch 4/50
17/17 [==============================] - 0s 8ms/step - loss: 2.0670 - accuracy: 0.3396 - val_loss: 2.1442 - val_accuracy: 0.3276
Epoch 5/50
17/17 [==============================] - 0s 8ms/step - loss: 1.8736 - accuracy: 0.3906 - val_loss: 2.1198 - val_accuracy: 0.3103
Epoch 6/50
17/17 [==============================] - 0s 7ms/step - loss: 1.6450 - accuracy: 0.4942 - val_loss: 2.1960 - val_accuracy: 0.2931
Epoch 7/50
17/17 [==============================] - 0s 7ms/step - loss: 1.5717 - accuracy: 0.5173 - val_loss: 2.1433 - val_accuracy: 0.3448
Epoch 8/50
17/17 [=

In [100]:
#Converting test images to array
def images_to_array2(data_dir,df, img_size):
    images_names = df['image']
    data_size = len(images_names)
    X = np.zeros([data_size, img_size[0], img_size[1], 3], dtype=np.uint8)
    
    for i in range(data_size):
        image_name = images_names[i]
        img_dir = os.path.join(data_dir, image_name)
        img_pixels = load_img(img_dir, target_size=img_size)
        X[i] = img_pixels
        
    print('Ouptut Data Size: ', X.shape)
    return X

In [102]:
sample_df = pd.read_csv('/content/drive/MyDrive/dataset/players.csv')

In [104]:
test_data = images_to_array2(train_dir, sample_df, img_size)

Ouptut Data Size:  (576, 299, 299, 3)


In [105]:
#Extract test data features.
inception_features = get_features(InceptionV3, inception_preprocessor, img_size, test_data)

18/18 [==============================] - 117s 6s/step
Feature maps shape:  (576, 2048)


In [106]:
xception_features = get_features(Xception, xception_preprocessor, img_size, test_data)

18/18 [==============================] - 218s 12s/step
Feature maps shape:  (576, 2048)


In [107]:
inc_resnet_features = get_features(InceptionResNetV2, inc_resnet_preprocessor, img_size, test_data)

18/18 [==============================] - 270s 15s/step
Feature maps shape:  (576, 1536)


In [108]:
test_features = np.concatenate([inception_features,
                                 xception_features,
                                 inc_resnet_features],axis=-1)
print('Final feature maps shape', test_features.shape)

Final feature maps shape (576, 5632)


In [109]:
del test_data

In [110]:
y_pred = model.predict(test_features, batch_size=32)

In [113]:
for breed in dog_breeds:
    sample_df[breed] = y_pred[:,class_to_num[breed]]
sample_df.to_csv('pred.csv', index=None)

In [114]:
sample_df.head()

,Unnamed: 0,image,player,bhuvneshwar_kumar,dinesh_karthik,hardik_pandya,jasprit_bumrah,k._l._rahul,kedar_jadhav,kuldeep_yadav,mohammed_shami,ms_dhoni,ravindra_jadeja,rohit_sharma,shikhar_dhawan,vijay_shankar,virat_kohli,yuzvendra_chahal
0,0,eda97f557b.jpg,virat_kohli,0.006527,0.001383,0.007624,0.009866,0.005711,0.002508,0.000173,0.000070,0.001737,0.020512,0.042944,0.005162,0.002011,0.893128,0.000644
1,1,273ed0f2b9.jpg,hardik_pandya,0.000703,0.000077,0.971743,0.000700,0.000214,0.002223,0.000083,0.001126,0.000130,0.002858,0.001626,0.005756,0.000815,0.011843,0.000104
2,2,77076b6409.jpg,jasprit_bumrah,0.000602,0.002045,0.001274,0.956341,0.002898,0.009200,0.001502,0.000826,0.000097,0.009110,0.000141,0.003315,0.007888,0.003468,0.001294
3,3,f028e2dcf8.jpg,mohammed_shami,0.001160,0.000068,0.006706,0.001743,0.001677,0.000136,0.000171,0.963331,0.000176,0.012029,0.000071,0.000032,0.005694,0.006208,0.000798
4,4,3364f861e1.jpg,kedar_jadhav,0.000479,0.001950,0.000336,0.002765,0.000259,0.985152,0.000970,0.000204,0.000057,0.001093,0.000207,0.000726,0.003785,0.000099,0.001918
